In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Insomnia"
cohort = "GSE208668"

# Input paths
in_trait_dir = "../../input/GEO/Insomnia"
in_cohort_dir = "../../input/GEO/Insomnia/GSE208668"

# Output paths
out_data_file = "../../output/preprocess/Insomnia/GSE208668.csv"
out_gene_data_file = "../../output/preprocess/Insomnia/gene_data/GSE208668.csv"
out_clinical_data_file = "../../output/preprocess/Insomnia/clinical_data/GSE208668.csv"
json_path = "../../output/preprocess/Insomnia/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains genome-wide transcriptional profiling data from PBMCs
# Note: Raw data was lost but processed data should be available for analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Insomnia)
trait_row = 0  # Key 0 contains 'insomnia: yes', 'insomnia: no'

# Age
age_row = 1  # Key 1 contains age information with multiple values (60-80)

# Gender
gender_row = 2  # Key 2 contains 'gender: female', 'gender: male'

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert insomnia status to binary format (0 for No, 1 for Yes)."""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'yes':
        return 1
    elif value == 'no':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to continuous format."""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary format (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Conduct initial filtering on dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical data to CSV
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers in the gene expression data
# The identifiers appear to be human gene symbols (like A1BG, A2M, etc.)
# These are standard gene symbols and not probe identifiers that would need mapping

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data - already done in previous step
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file - already done in previous step
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Load the clinical features from the previous step
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(selected_clinical_df.head())

# Link the clinical and genetic data directly using the properly encoded clinical data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data related to alcohol dependence but was evaluated for Schizophrenia."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")